In [1]:
import os
os.chdir("../..")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset_loader import *
from utils.model_utils import *
from utils.train_utils import *
from utils.metrics import *
from utils.visualization import *
import optuna
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

In [5]:
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(
    "wildfire_dataset_scaled", batch_size=8, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")

2024-12-24 14:32:50,924 - INFO - Loading datasets from wildfire_dataset_scaled with augmentation type 'augmented'.
2024-12-24 14:32:50,925 - INFO - Applying selected augmentations for wildfire scenarios.
2024-12-24 14:32:50,939 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/train
2024-12-24 14:32:50,943 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/val
2024-12-24 14:32:50,946 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/test
2024-12-24 14:32:50,947 - INFO - Datasets initialized. Preparing DataLoaders...
2024-12-24 14:32:50,948 - INFO - Using WeightedRandomSampler for class balancing.
2024-12-24 14:32:50,948 - INFO - Computing class weights from directory: wildfire_dataset_scaled/train
2024-12-24 14:32:50,950 - INFO - Class 'fire' has 730 samples.
2024-12-24 14:32:50,952 - INFO - Class 'nofire' has 1157 samples.
2024-12-24 14:32:50,953 - INFO - Computed class weights: {'fire': 1.0, 'nofire': 

[INFO] Loading datasets...


2024-12-24 14:33:04,916 - INFO - DataLoaders created successfully.


[INFO] Datasets loaded successfully!


In [6]:
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    print("[DEBUG] Initializing convnext_tiny model...")
    model = initialize_model("convnext_tiny", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=12)
    model.to(device)

    # Optimizer setup
    print("[DEBUG] Setting up optimizer...")
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = "outputs/models/tuned/convnext_tiny_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=5,
        monitor_metric="val_f1",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_f1"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation F1: {val_recall:.4f}")
    return val_recall


In [ ]:
# Run Optuna
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters found: {study.best_params}")

[I 2024-12-24 14:33:04,936] A new study created in memory with name: no-name-e69d4079-32c3-4adf-8d2c-f05d6b574358


[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[DEBUG] Suggested hyperparameters: lr=7.565800971824634e-05, weight_decay=0.0004061072493103097, optimizer=Adam
[DEBUG] Initializing convnext_tiny model...
[DEBUG] Setting up optimizer...
[INFO] Starting model training...

Starting training...



[INFO] Learning rate adjusted to: 0.000076
[INFO] Best model saved with val_f1: 0.8809
Epoch [1]: Train Loss: 0.3205, Train Acc: 0.8649 | Val Loss: 0.5673, Val Acc: 0.8358, Val Recall: 0.9919, Val F1: 0.8809


[INFO] Learning rate adjusted to: 0.000076
[INFO] Best model saved with val_f1: 0.9412
Epoch [2]: Train Loss: 0.1501, Train Acc: 0.9571 | Val Loss: 0.2027, Val Acc: 0.9279, Val Recall: 0.9431, Val F1: 0.9412


[INFO] Learning rate adjusted to: 0.000076
[INFO] No improvement in val_f1. Patience: 1/5
Epoch [3]: Train Loss: 0.1261, Train Acc: 0.9560 | Val Loss: 0.2291, Val Acc: 0.9254, Val Recall: 0.9228, Val F1: 0.9380


...

[INFO] Learning rate adjusted to: 0.000031
[INFO] Best model saved with val_f1: 0.9416
Epoch [3]: Train Loss: 0.1071, Train Acc: 0.9650 | Val Loss: 0.2150, Val Acc: 0.9279, Val Recall: 0.9512, Val F1: 0.9416


[INFO] Learning rate adjusted to: 0.000031
[INFO] No improvement in val_f1. Patience: 1/5
Epoch [4]: Train Loss: 0.0813, Train Acc: 0.9730 | Val Loss: 0.2653, Val Acc: 0.9204, Val Recall: 0.9756, Val F1: 0.9375


[INFO] Learning rate adjusted to: 0.000031
[INFO] No improvement in val_f1. Patience: 2/5
Epoch [5]: Train Loss: 0.0598, Train Acc: 0.9783 | Val Loss: 0.2296, Val Acc: 0.9279, Val Recall: 0.9268, Val F1: 0.9402


[INFO] Learning rate adjusted to: 0.000031
[INFO] Best model saved with val_f1: 0.9514
Epoch [6]: Train Loss: 0.0459, Train Acc: 0.9846 | Val Loss: 0.2747, Val Acc: 0.9403, Val Recall: 0.9553, Val F1: 0.9514


[INFO] Learning rate adjusted to: 0.000031
[INFO] No improvement in val_f1. Patience: 1/5
Epoch [7]: Train Loss: 0.0452, Train Acc: 0.9894 | Val Loss: 0.2524, Val Acc: 0.9303, Val Recall: 0.9797, Val F1: 0.9451


In [ ]:
# Train Final Model
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model("convnext_tiny", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=12)
final_model.to(device)

In [ ]:
# Optimizer
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params["lr"], weight_decay=study.best_params["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

In [ ]:
# Final training
final_save_path = "outputs/models/tuned/convnext_tiny_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=5,
    monitor_metric="val_f1",
)

In [ ]:
# Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device, model_name="convnext_tiny", save_base_path="outputs/plots/tuned/convnexttiny")
# Training and Validation Curves
plot_training(history, f"outputs/plots/tuned/convnextnet_tuned_training_curve.png")

# Confusion Matrix
plot_confusion_matrix(
    cm=metrics["confusion_matrix"],
    classes=["No Fire", "Fire"],
    output_path=f"outputs/plots/tuned/convnextnet_tuned_confusion_matrix.png"
)

# Precision-Recall Curve
if "y_probs" in metrics:
    y_true = metrics["y_true"]
    y_probs = [prob[1] for prob in metrics["y_probs"]]
    plot_precision_recall(
        y_true=y_true,
        y_scores=y_probs,
        output_path=f"outputs/plots/tuned/convnextnet_tuned_precision_recall_curve.png"
    )

    # ROC Curve
    plot_roc_curve(
        y_true=y_true,
        y_scores=y_probs,
        output_path=f"outputs/plots/tuned/convnextnet_tuned_roc_curve.png"
    )

print("[INFO] All results saved successfully!")